# Lisbon's Public Transport Network Analysis

In this project, I built a real-time data pipeline to monitor public bus statuses using a live API. The goal was to simulate an end-to-end data workflow — from raw data collection to interactive dashboards.

Python collects and cleans real-time bus data from a public API (e.g., “In Transit”, “Stationed”).

Cleaned data is sent to SQL Server, which acts as the central storage.

SQL is used for hypothesis testing and querying traffic patterns.

Power BI connects to SQL and displays a live dashboard.

The refresh button in Power BI re-triggers the pipeline:
Python pulls → SQL updates → Power BI refreshes.

🔧 Key Skills Demonstrated
API integration & real-time data handling

Data cleaning and transformation with Python

SQL for analysis & database management

Power BI dashboard creation

End-to-end automation mindset

This project shows not just visualization — but a working pipeline from data ingestion to insights.

1. **Data Collection**
   - Real-time bus positions
   - Metro line status monitoring

2. **Analysis Features**
   - Geographic visualization
   - District-based analysis
   - Status monitoring
   - Performance metrics

## 1. Setup and Dependencies
First, let's import all the required libraries for our analysis

In [1]:
import requests
import json
import pandas as pd
from tabulate import tabulate
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pyodbc
import gtfs_realtime_pb2
import google.protobuf


### System Architecture

1. **Data Collection Layer** 📥
   - Real-time bus position tracking
   - Integration with multiple sources:
     - Carris (traditional bus system)
     - Carris Metropolitana (metropolitan area)
   - GTFS Realtime format processing



- Fetch Real time Lisbon Carris Trasnport

In [2]:
Carris_CL = None

def get_all_vehicle_data():
    global Carris_CL
    url = "https://gateway.carris.pt/gateway/gtfs/api/v2.11/GTFS/realtime/vehiclepositions"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to get data: {response.status_code}")
        Carris_CL = None
        return

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(response.content)

    data_list = []

    def parse_message(message, prefix=''):
        result = {}
        for field, value in message.ListFields():
            name = f"{prefix}{field.name}"
            if hasattr(value, 'ListFields'):
                result.update(parse_message(value, prefix=name+'_'))
            else:
                result[name] = value
        return result

    for entity in feed.entity:
        if entity.HasField('vehicle'):
            vehicle_data = parse_message(entity.vehicle)
            vehicle_data['entity_id'] = entity.id
            data_list.append(vehicle_data)

    if data_list:
        Carris_CL = pd.DataFrame(data_list)
        
        # Mapeamento dos status
        status_map = {
            0: 'INCOMING_AT',
            1: 'STOPPED_AT',
            2: 'IN_TRANSIT_TO'
        }
        
     
        if 'current_status' in Carris_CL.columns:
            Carris_CL['current_status_text'] = Carris_CL['current_status'].map(status_map)
        else:
            print("Coluna 'current_status' não encontrada no DataFrame.")
        
        print(f"Got {len(Carris_CL)} vehicles with {len(Carris_CL.columns)} columns.")
        print(tabulate(Carris_CL.head(), headers='keys', tablefmt='grid', showindex=False))
    else:
        print("No vehicle data found.")
        Carris_CL = None


# Call function to update Carris_CL
get_all_vehicle_data()


Got 407 vehicles with 14 columns.
+-----------------------+------------------------------+-----------------+---------------------+---------------------+----------------------+-------------------------+------------------+-------------+-----------+--------------+-------------------------+-------------+-----------------------+
|          trip_trip_id |   trip_schedule_relationship |   trip_route_id |   trip_direction_id |   position_latitude |   position_longitude |   current_stop_sequence |   current_status |   timestamp |   stop_id |   vehicle_id | vehicle_license_plate   | entity_id   | current_status_text   |
+=======================+==============================+=================+=====================+=====================+======================+=========================+==================+=============+===========+==============+=========================+=============+=======================+
| 5094_20250601_107_0_4 |                            0 |           107_0 |                

2. **Processing Layer** 🔄
   - Geographic data enrichment
   - Status monitoring & classification
   - Multi-source data integration
   - Performance metrics calculation

In [3]:
# rename columns
Carris_CL.rename(columns={
    'entity_id': 'Line_ID',
    'timestamp': 'Timestamp',
    'position_latitude': 'Latitude',
    'position_longitude': 'Longitude',
    'current_status_text': 'Current_Status',
    'trip_route_id': 'Route_ID',
    'stop_id': 'Stop_ID',
    'trip_trip_id': 'Trip_ID',
    'trip_direction_id': 'Direction_ID'
}, inplace=True)

# Replace values in the 'Current_Status' column
Carris_CL['Current_Status'] = Carris_CL['Current_Status'].replace({
    'IN_TRANSIT_TO': 'In Transit',
    'INCOMING_AT': 'Approaching',
    'STOPPED_AT': 'Stationed'
})

# Keep last 3 characters of Line_ID
Carris_CL['Line_ID'] = Carris_CL['Line_ID'].str[-3:]

print(f"Got {len(Carris_CL)} vehicles with {len(Carris_CL.columns)} columns.")
print(tabulate(Carris_CL.head(), headers='keys', tablefmt='grid', showindex=False))



Got 407 vehicles with 14 columns.
+-----------------------+------------------------------+------------+----------------+------------+-------------+-------------------------+------------------+-------------+-----------+--------------+-------------------------+-----------+------------------+
|               Trip_ID |   trip_schedule_relationship |   Route_ID |   Direction_ID |   Latitude |   Longitude |   current_stop_sequence |   current_status |   Timestamp |   Stop_ID |   vehicle_id | vehicle_license_plate   | Line_ID   | Current_Status   |
+=======================+==============================+============+================+============+=============+=========================+==================+=============+===========+==============+=========================+===========+==================+
| 5094_20250601_107_0_4 |                            0 |      107_0 |              0 |    38.7215 |    -9.14675 |                      13 |                2 |  1753780604 |      1212 |         60

- TimesTamp Converter

In [4]:

if 'Timestamp' in Carris_CL.columns:
    # Converte timestamp para datetime e extrai componentes
    Carris_CL.loc[:, 'Datetime'] = pd.to_datetime(Carris_CL['Timestamp'], unit='s')
    Carris_CL.loc[:, 'Date'] = Carris_CL['Datetime'].dt.strftime('%Y-%m-%d')
    Carris_CL.loc[:, 'Time'] = Carris_CL['Datetime'].dt.strftime('%H:%M:%S')
    Carris_CL.loc[:, 'Day'] = Carris_CL['Datetime'].dt.day_name()
    Carris_CL.loc[:, 'Workday'] = Carris_CL['Datetime'].dt.dayofweek.apply(lambda x: x < 5)

    # Mostra o horário atual
    print(f"\nCurrent time: {pd.Timestamp.now()}")

    # Exibe resultados
    print("\nProcessed datetime information:")
    # Display the final DataFrame
    print(tabulate(Carris_CL.head(),
               headers='keys',
               tablefmt='grid',
               showindex=False,
               floatfmt='.6f'))
else:
    print("Error: 'Timestamp' column not found in Carris_CL DataFrame.")


Current time: 2025-07-29 10:17:12.918903

Processed datetime information:
+-----------------------+------------------------------+------------+----------------+------------+-------------+-------------------------+------------------+-------------+-----------+--------------+-------------------------+-----------+------------------+---------------------+------------+----------+---------+-----------+
|               Trip_ID |   trip_schedule_relationship |   Route_ID |   Direction_ID |   Latitude |   Longitude |   current_stop_sequence |   current_status |   Timestamp |   Stop_ID |   vehicle_id | vehicle_license_plate   | Line_ID   | Current_Status   | Datetime            | Date       | Time     | Day     | Workday   |
+=======================+==============================+============+================+============+=============+=========================+==================+=============+===========+==============+=========================+===========+==================+===================

- Add Column Color

In [5]:
# Load the CSV Color_ID

color_df = pd.read_csv(r"D:\Project\Lisbon’s Public Transport Network\Datasets\Line_Colors.csv")

color_df.columns = color_df.columns.str.strip()

# 2. Renomear a coluna de chave para 'Line_ID' para facilitar o merge
color_df.rename(columns={'LineID': 'Line_ID'}, inplace=True)

# Merge Carris_CL with color_df on 'Line_ID'
Carris_CL = Carris_CL.merge(
    color_df[['Line_ID', 'Color_ID', 'Operator']],
    on='Line_ID',
    how='left'
)

from tabulate import tabulate

print(tabulate(Carris_CL.head(), headers='keys', tablefmt='grid', showindex=False, floatfmt=".6f"))

+-----------------------+------------------------------+------------+----------------+------------+-------------+-------------------------+------------------+-------------+-----------+--------------+-------------------------+-----------+------------------+---------------------+------------+----------+---------+-----------+-------------+------------+
|               Trip_ID |   trip_schedule_relationship |   Route_ID |   Direction_ID |   Latitude |   Longitude |   current_stop_sequence |   current_status |   Timestamp |   Stop_ID |   vehicle_id | vehicle_license_plate   | Line_ID   | Current_Status   | Datetime            | Date       | Time     | Day     | Workday   | Color_ID    | Operator   |
+=======================+==============================+============+================+============+=============+=========================+==================+=============+===========+==============+=========================+===========+==================+=====================+============+=====

## Project Features

### 1. Data Collection 🔄
- Real-time vehicle positions
- Multiple operator integration
- Automated data updates

- Fetch Real time Lisbon Carris Metropolitana Transport


In [6]:

Carris_CM = None  # global variable

def get_realtime_bus_positions():
    global Carris_CM

    url = "https://api.carrismetropolitana.pt/v1/vehicles"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if data:
            Carris_CM = pd.DataFrame(data)
            print(f"\nFound {len(Carris_CM)} vehicles. Showing first 5:")
            print(tabulate(Carris_CM.head(), headers='keys', tablefmt='grid', showindex=False))
        else:
            print("No vehicles found.")
            Carris_CM = None
            
    except requests.RequestException as e:
        print("Error fetching data:", e)
        Carris_CM = None

# Call the function to update Carris_CM
get_realtime_bus_positions()




Found 818 vehicles. Showing first 5:
+-----------+--------------------------------+------------------+----------+---------+-----------+----------+--------------+------------+-------------------------+--------------+----------+-----------+-------------+--------------------------+
|   bearing | block_id                       | current_status   | id       |     lat |   line_id |      lon |   pattern_id |   route_id | schedule_relationship   |     shift_id |    speed |   stop_id |   timestamp | trip_id                  |
+===========+================================+==================+==========+=========+===========+==========+==============+============+=========================+==============+==========+===========+=============+==========================+
|       183 | 20250729-64810400-313190234560 | IN_TRANSIT_TO    | 44|12747 | 38.7085 |      4730 | -9.17391 |     4730_0_1 |     4730_0 | SCHEDULED               | 313190234560 |  0       |    020001 |  1753780614 | 4730_0_1|3700|100

### 2.1 Data Processing and Column Renaming
Process the raw vehicle data and rename columns for better readability. This step makes the data more intuitive to work with by:
- Renaming columns with more descriptive names
- Standardizing column naming conventions

In [7]:

# Convert vehicle_data to a DataFrame
Carris_CM = pd.DataFrame(Carris_CM)

# Rename the columns for better readability
def rename_columns(Carris_CM):
    Carris_CM.rename(columns={
        'id': 'Vehicle ID',
        'line_id': 'Line_ID',
        'lat': 'Latitude',
        'lon': 'Longitude',
        'route_id': 'Route_ID',
        'current_status': 'Current_Status',
        'bearing': 'Direction_ID',
        'pattern_id': 'Pattern_ID',
        'speed': 'Speed',
        'timestamp': 'Timestamp',
        'shift_id': 'Shift_ID',
        'stop_id': 'Stop_ID',
        'trip_id': 'Trip_ID',
        'block_id': 'Travel_ID',
    }, inplace=True)
    return Carris_CM

# Rename columns
Carris_CM = rename_columns(Carris_CM)


# Drop Vehicle ID column
#Carris_CM.drop(columns=['Vehicle ID'], inplace=True)

# Replace values in the 'Current_Status' column
Carris_CM['Current_Status'] = Carris_CM['Current_Status'].replace({
    'IN_TRANSIT_TO': 'In Transit',
    'INCOMING_AT': 'Approaching',
    'STOPPED_AT': 'Stationed'
})


print(f"\nFound {len(Carris_CM)} vehicles. Showing first 5:")
print(tabulate(Carris_CM.head(), headers='keys', tablefmt='grid', showindex=False))


Found 818 vehicles. Showing first 5:
+----------------+--------------------------------+------------------+--------------+------------+-----------+-------------+--------------+------------+-------------------------+--------------+----------+-----------+-------------+--------------------------+
|   Direction_ID | Travel_ID                      | Current_Status   | Vehicle ID   |   Latitude |   Line_ID |   Longitude |   Pattern_ID |   Route_ID | schedule_relationship   |     Shift_ID |    Speed |   Stop_ID |   Timestamp | Trip_ID                  |
+================+================================+==================+==============+============+===========+=============+==============+============+=========================+==============+==========+===========+=============+==========================+
|            183 | 20250729-64810400-313190234560 | In Transit       | 44|12747     |    38.7085 |      4730 |    -9.17391 |     4730_0_1 |     4730_0 | SCHEDULED               | 313190234

In [8]:
# Carris_CM: process datetime
if 'Timestamp' in Carris_CM.columns:
    Carris_CM.loc[:, 'Datetime'] = pd.to_datetime(Carris_CM['Timestamp'], unit='s')
    Carris_CM.loc[:, 'Date'] = Carris_CM['Datetime'].dt.strftime('%Y-%m-%d')
    Carris_CM.loc[:, 'Time'] = Carris_CM['Datetime'].dt.strftime('%H:%M:%S')
    Carris_CM.loc[:, 'Day'] = Carris_CM['Datetime'].dt.day_name()
    Carris_CM.loc[:, 'Workday'] = Carris_CM['Datetime'].dt.dayofweek.apply(lambda x: x < 5)

  # Exibe resultados
    print("\nProcessed datetime information:")
    # Display the final DataFrame
    print(tabulate(Carris_CM.head(),
               headers='keys',
               tablefmt='grid',
               showindex=False,
               floatfmt='.6f'))


Processed datetime information:
+----------------+--------------------------------+------------------+--------------+------------+-----------+-------------+--------------+------------+-------------------------+--------------+-----------+-----------+-------------+--------------------------+---------------------+------------+----------+---------+-----------+
|   Direction_ID | Travel_ID                      | Current_Status   | Vehicle ID   |   Latitude |   Line_ID |   Longitude |   Pattern_ID |   Route_ID | schedule_relationship   |     Shift_ID |     Speed |   Stop_ID |   Timestamp | Trip_ID                  | Datetime            | Date       | Time     | Day     | Workday   |
+================+================================+==================+==============+============+===========+=============+==============+============+=========================+==============+===========+===========+=============+==========================+=====================+============+==========+=========

In [9]:
# Load the CSV Color_ID

color_df = pd.read_csv(r"D:\Project\Lisbon’s Public Transport Network\Datasets\Line_Colors.csv")

color_df.columns = color_df.columns.str.strip()

# 2. Renomear a coluna de chave para 'Line_ID' para facilitar o merge
color_df.rename(columns={'LineID': 'Line_ID'}, inplace=True)

# Merge Carris_CL with color_df on 'Line_ID'
Carris_CM = Carris_CM.merge(
    color_df[['Line_ID', 'Color_ID', 'Operator']],
    on='Line_ID',
    how='left'
)

from tabulate import tabulate

print(tabulate(Carris_CM.head(), headers='keys', tablefmt='grid', showindex=False, floatfmt=".6f"))

+----------------+--------------------------------+------------------+--------------+------------+-----------+-------------+--------------+------------+-------------------------+--------------+-----------+-----------+-------------+--------------------------+---------------------+------------+----------+---------+-----------+-------------+------------+
|   Direction_ID | Travel_ID                      | Current_Status   | Vehicle ID   |   Latitude |   Line_ID |   Longitude |   Pattern_ID |   Route_ID | schedule_relationship   |     Shift_ID |     Speed |   Stop_ID |   Timestamp | Trip_ID                  | Datetime            | Date       | Time     | Day     | Workday   | Color_ID    | Operator   |
+================+================================+==================+==============+============+===========+=============+==============+============+=========================+==============+===========+===========+=============+==========================+=====================+============

In [10]:
print("CM Columns:", Carris_CM.columns.tolist())
print("CL Columns:", Carris_CL.columns.tolist())

CM Columns: ['Direction_ID', 'Travel_ID', 'Current_Status', 'Vehicle ID', 'Latitude', 'Line_ID', 'Longitude', 'Pattern_ID', 'Route_ID', 'schedule_relationship', 'Shift_ID', 'Speed', 'Stop_ID', 'Timestamp', 'Trip_ID', 'Datetime', 'Date', 'Time', 'Day', 'Workday', 'Color_ID', 'Operator']
CL Columns: ['Trip_ID', 'trip_schedule_relationship', 'Route_ID', 'Direction_ID', 'Latitude', 'Longitude', 'current_stop_sequence', 'current_status', 'Timestamp', 'Stop_ID', 'vehicle_id', 'vehicle_license_plate', 'Line_ID', 'Current_Status', 'Datetime', 'Date', 'Time', 'Day', 'Workday', 'Color_ID', 'Operator']


## Combining the dataset

In [11]:
# Step 1: Identify and add missing columns from Carris_CM into Carris_CL
for col in Carris_CM.columns:
    if col not in Carris_CL.columns:
        Carris_CL[col] = None  # or use np.nan if you prefer float-compatible missing values

# Step 2: Reorder columns in Carris_CL to match the order in Carris_CM
Carris_CL = Carris_CL[Carris_CM.columns]

# Step 3 (optional but recommended): Add a 'Source' column to track origin
Carris_CM['Source'] = 'CM'
Carris_CL['Source'] = 'CL'

# Step 4: Add 'Source' to columns list to match before concatenation
common_columns = Carris_CM.columns  # Now includes the 'Source'

# Step 5: Concatenate the aligned DataFrames
Bus_System = pd.concat([Carris_CM, Carris_CL], ignore_index=True)[common_columns]

# Step 6: Print result for confirmation
print(f"Bus_System shape: {Bus_System.shape}")
print(f"Total rows: {len(Bus_System)}")
print(f"Total columns: {len(Bus_System.columns)}")
print("CM Columns:", Carris_CM.columns.tolist())
print("CL Columns:", Carris_CL.columns.tolist())
print(tabulate(Bus_System.head(), headers='keys', tablefmt='grid', showindex=False))


Bus_System shape: (1226, 23)
Total rows: 1226
Total columns: 23
CM Columns: ['Direction_ID', 'Travel_ID', 'Current_Status', 'Vehicle ID', 'Latitude', 'Line_ID', 'Longitude', 'Pattern_ID', 'Route_ID', 'schedule_relationship', 'Shift_ID', 'Speed', 'Stop_ID', 'Timestamp', 'Trip_ID', 'Datetime', 'Date', 'Time', 'Day', 'Workday', 'Color_ID', 'Operator', 'Source']
CL Columns: ['Direction_ID', 'Travel_ID', 'Current_Status', 'Vehicle ID', 'Latitude', 'Line_ID', 'Longitude', 'Pattern_ID', 'Route_ID', 'schedule_relationship', 'Shift_ID', 'Speed', 'Stop_ID', 'Timestamp', 'Trip_ID', 'Datetime', 'Date', 'Time', 'Day', 'Workday', 'Color_ID', 'Operator', 'Source']
+----------------+--------------------------------+------------------+--------------+------------+-----------+-------------+--------------+------------+-------------------------+--------------+----------+-----------+-------------+--------------------------+---------------------+------------+----------+---------+-----------+-------------+---

C:\Users\cleid\AppData\Local\Temp\ipykernel_29100\840636999.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Bus_System = pd.concat([Carris_CM, Carris_CL], ignore_index=True)[common_columns]


### 2.3 Geographic Enrichment
Enrich the bus position data with district information using reverse geocoding. This process:
- Uses the Nominatim geocoding service
- Implements rate limiting to avoid API restrictions
- Adds district names based on latitude and longitude
- Handles potential geocoding errors gracefully

In [12]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tabulate import tabulate
import joblib
import os

# ========== 1. Geocoder setup and cache ==========
geolocator = Nominatim(user_agent="lisbon-bus-project")
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
CACHE_FILE = "district_cache.pkl"

# Load cache if available
coord_to_district = joblib.load(CACHE_FILE) if os.path.exists(CACHE_FILE) else {}

# ========== 2. Reusable function ==========
def assign_district(df, name="DataFrame"):
    """
    Takes a DataFrame with 'Latitude' and 'Longitude' columns,
    adds a 'District' column using cached reverse geocoding.
    """
    def get_district(lat, lon):
        coord = (round(lat, 4), round(lon, 4))  # Rounding to reduce unnecessary geocoding
        if coord in coord_to_district:
            return coord_to_district[coord]
        try:
            location = geocode(coord, exactly_one=True)
            if location and location.raw.get('address'):
                address = location.raw['address']
                district = (
                    address.get('suburb') or
                    address.get('neighbourhood') or
                    address.get('city_district') or
                    address.get('town') or
                    'Unknown'
                )
            else:
                district = 'Unknown'
        except Exception as e:
            print(f"Error geocoding {coord}: {e}")
            district = 'Unknown'
        coord_to_district[coord] = district
        return district

    # Apply geocoding only to unique coordinates
    unique_coords = df[['Latitude', 'Longitude']].drop_duplicates()
    unique_coords['District'] = unique_coords.apply(
        lambda row: get_district(row['Latitude'], row['Longitude']), axis=1
    )

    # Merge districts back into the original DataFrame
    df = df.merge(unique_coords, on=['Latitude', 'Longitude'], how='left')

    # Save updated cache
    joblib.dump(coord_to_district, CACHE_FILE)

    print(f"[✔] '{name}' updated with 'District' column.")
    return df

# ========== 3. Apply to your two DataFrames ==========
Carris_CM = assign_district(Carris_CM, name="Carris_CM")
Carris_CL = assign_district(Carris_CL, name="Carris_CL")

# Optional: Show preview
print(tabulate(Carris_CM.head(), headers='keys', tablefmt='grid', showindex=False, floatfmt=".6f"))


RateLimiter caught an error, retrying (0/2 tries). Called with (*((np.float64(38.6462), np.float64(-9.2337)),), **{'exactly_one': True}).
Traceback (most recent call last):
  File "d:\Project\Lisbon’s Public Transport Network\.venv\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "d:\Project\Lisbon’s Public Transport Network\.venv\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\cleid\AppData\Local\Programs\Python\Python313\Lib\http\client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "C:\Users\cleid\AppData\Local\Programs\Python\Python313\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "C:\Users\cleid\AppData\Local\Programs\Python\Python313\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.re

[✔] 'Carris_CM' updated with 'District' column.
[✔] 'Carris_CL' updated with 'District' column.
+----------------+--------------------------------+------------------+--------------+------------+-----------+-------------+--------------+------------+-------------------------+--------------+-----------+-----------+-------------+--------------------------+---------------------+------------+----------+---------+-----------+-------------+------------+----------+--------------------+
|   Direction_ID | Travel_ID                      | Current_Status   | Vehicle ID   |   Latitude |   Line_ID |   Longitude |   Pattern_ID |   Route_ID | schedule_relationship   |     Shift_ID |     Speed |   Stop_ID |   Timestamp | Trip_ID                  | Datetime            | Date       | Time     | Day     | Workday   | Color_ID    | Operator   | Source   | District           |
+================+================================+==================+==============+============+===========+=============+========

In [13]:
# combine the two DataFrames
Bus_System = pd.concat([Carris_CL, Carris_CM], ignore_index=True)

# save to csv   
Bus_System.to_csv(r"D:\Project\Lisbon’s Public Transport Network\Datasets\Bus_System.csv", index=False)

C:\Users\cleid\AppData\Local\Temp\ipykernel_29100\400385356.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Bus_System = pd.concat([Carris_CL, Carris_CM], ignore_index=True)


## Separete DataSet 

In [14]:
# Include both 'E' and 'B' in the Tram group
Tram_df = Bus_System[
    Bus_System['Line_ID'].astype(str).str.contains('E|B', case=False, na=False)
]

# Exclude both 'E' and 'B' for the Bus group
Bus_df = Bus_System[
    ~Bus_System['Line_ID'].astype(str).str.contains('E|B', case=False, na=False)
]

# Save to CSV
Tram_df.to_csv(r"D:\Project\Lisbon’s Public Transport Network\Datasets\tram_data.csv", index=False)
Bus_df.to_csv(r"D:\Project\Lisbon’s Public Transport Network\Datasets\bus_data.csv", index=False)
